# Proof Of Concept

This notebook aims to apprehend the topic and structure the goal of this project

## Librairies

In [137]:
import pandas as pd
import yfinance as yf
import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import functools as ft
import networkx as nx
import numpy as np

import torch
from torch_geometric.data import Data, Batch

## 1. Prepare Data

### Extract

In [14]:
t = yf.Ticker("PETR4.SA")
petr4 = t.history(period="max").reset_index()[["Date", "Close"]]
petr4["Date"] = petr4['Date'].dt.strftime('%Y%m%d')
petr4.columns = ["Date", "petr4_close"]

In [26]:
t = yf.Ticker("BPAC11.SA")
btg = t.history(period="max").reset_index()[["Date", "Close"]]
btg["Date"] = btg['Date'].dt.strftime('%Y%m%d')
btg.columns = ["Date", "btg_close"]

In [27]:
t = yf.Ticker("B3SA3.SA")
b3 = t.history(period="max").reset_index()[["Date", "Close"]]
b3["Date"] = b3['Date'].dt.strftime('%Y%m%d')
b3.columns = ["Date", "b3_close"]

### Join

In [29]:
df = ft.reduce(lambda left, right: pd.merge(left, right, on='Date', how = "inner"), [petr4, btg, b3])

In [62]:
dataset = df.sort_values("Date").drop(columns="Date")

In [77]:
dataset

,petr4_close,btg_close,b3_close
0,5.587599,3.654601,5.058690
1,5.712750,3.604400,5.079965
2,5.561833,3.604400,4.973579
3,5.639132,3.765042,5.000175
4,5.558152,3.855403,4.968258
...,...,...,...
1590,29.100000,32.889999,14.600000
1591,29.129999,33.320000,14.610000
1592,29.680000,34.080002,15.170000
1593,30.299999,33.700001,15.100000


In [80]:
6*1586

9516

## Create dataset

### Training set

In [122]:
# Create a mapping of node names (stocks) to integers
node_mapping = {node: i for i, node in enumerate(dataset.columns)}

edge_indices = []
edge_weights = []
node_features = []
graphs = []
labels = []

k = 10
test_size = 100

for t in tqdm(range(k, len(dataset) - test_size)):
    
    # Step 1: Create correlation matrix and graph for this time step
    correlation_matrix = dataset.iloc[:t].corr()
    graph = nx.from_pandas_adjacency(correlation_matrix)
    graphs.append(graph)

    # Step 2: Convert edges to integer tuples and store as tensor
    edges = list(graph.edges)
    edges_mapped = [(node_mapping[u], node_mapping[v]) for u, v in edges]
    edge_indices.append(torch.tensor(edges_mapped, dtype=torch.long).t().contiguous())

    # Step 3: Store edge weights as tensor
    edge_weights.append(torch.tensor([graph[u][v]['weight'] for u, v in edges], dtype=torch.float))

    # Step 4: Store closing prices as node features (assume last column of df is 'Close')
    node_features.append(torch.tensor(dataset.iloc[t-k:t, :].T.values, dtype=torch.float))
    
    # Step 5: Store prices at t+1 as labels
    labels.append(torch.tensor(dataset.iloc[t, :], dtype=torch.float))

# Combine edge_indices, edge_weights, and node_features into a list of Data objects
data_list = [Data(x=node_features[i], edge_index=edge_indices[i], edge_attr=edge_weights[i], y = labels[i]) for i in range(len(graphs))]

# Convert data_list into a Batch for feeding into your model
batch = Batch.from_data_list(data_list)


100%|██████████| 1485/1485 [00:01<00:00, 796.66it/s]


### Test set

In [142]:
# Create a mapping of node names (stocks) to integers
node_mapping = {node: i for i, node in enumerate(dataset.columns)}

edge_indices = []
edge_weights = []
node_features = []
graphs = []
labels = []

k = 10
test_size = 100

for t in tqdm(range(len(dataset) - test_size, len(dataset))):
    
    # Step 1: Create correlation matrix and graph for this time step
    correlation_matrix = dataset.iloc[:t].corr()
    graph = nx.from_pandas_adjacency(correlation_matrix)
    graphs.append(graph)

    # Step 2: Convert edges to integer tuples and store as tensor
    edges = list(graph.edges)
    edges_mapped = [(node_mapping[u], node_mapping[v]) for u, v in edges]
    edge_indices.append(torch.tensor(edges_mapped, dtype=torch.long).t().contiguous())

    # Step 3: Store edge weights as tensor
    edge_weights.append(torch.tensor([graph[u][v]['weight'] for u, v in edges], dtype=torch.float))

    # Step 4: Store closing prices as node features (assume last column of df is 'Close')
    node_features.append(torch.tensor(dataset.iloc[t-k:t, :].T.values, dtype=torch.float))
    
    # Step 5: Store prices at t+1 as labels
    labels.append(torch.tensor(dataset.iloc[t, :], dtype=torch.float))

# Combine edge_indices, edge_weights, and node_features into a list of Data objects
data_list_test = [Data(x=node_features[i], edge_index=edge_indices[i], edge_attr=edge_weights[i], y = labels[i]) for i in range(len(graphs))]

# Convert data_list into a Batch for feeding into your model
batch_test = Batch.from_data_list(data_list_test)


100%|██████████| 100/100 [00:00<00:00, 518.17it/s]


## Model 

In [123]:
import torch.nn.functional as F
from torch_geometric.nn import GraphConv
from torch.optim import Adam

class GNN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GNN, self).__init__()
        self.conv1 = GraphConv(num_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        return x


## Training

In [127]:
# Assuming node features are 3 and we want to predict next price (1 value)
model = GNN(num_features=10, hidden_channels=6, num_classes=1)

In [136]:

# Define a suitable optimizer
optimizer = Adam(model.parameters(), lr=0.01)

# Define loss function - mean squared error loss
loss_func = torch.nn.MSELoss()

# Number of epochs
epochs = 100

In [138]:
# Training loop
for epoch in range(epochs):
    loss_list = []
    print("EPOCH > ", epoch)
    for data in tqdm(batch.to_data_list()): # Iterate over each graph in the batch
        model.train()
        optimizer.zero_grad()

        #print("data: ", data)
        # Forward pass
        out = model(data).squeeze()
        
        #print("out: ", out)

        # Calculate loss
        loss = loss_func(out, data.y)
        loss_list.append(loss.item())
        
        
        # Backward pass
        loss.backward()
        optimizer.step()

        model.eval()
    print("Mean Loss: ", np.mean(loss_list))


EPOCH >  0


100%|██████████| 1485/1485 [00:01<00:00, 923.46it/s]


Mean Loss:  0.5580933461223669
EPOCH >  1


100%|██████████| 1485/1485 [00:01<00:00, 962.38it/s]


Mean Loss:  0.6621371364200594
EPOCH >  2


...

Mean Loss:  0.49384914197722257
EPOCH >  98


100%|██████████| 1485/1485 [00:01<00:00, 824.91it/s]


Mean Loss:  0.49377981824980677
EPOCH >  99


100%|██████████| 1485/1485 [00:01<00:00, 935.59it/s] 

Mean Loss:  0.4940819965024802


### Testing

In [143]:
N = len(dataset)
testset = dataset.iloc[N-test_size:, :]

In [144]:
testset

,petr4_close,btg_close,b3_close
1495,21.133572,19.440001,10.355125
1496,20.582260,19.379999,10.443884
1497,21.467701,19.320000,10.374849
1498,21.684881,19.780001,10.808779
1499,20.966507,19.410000,10.897537
...,...,...,...
1590,29.100000,32.889999,14.600000
1591,29.129999,33.320000,14.610000
1592,29.680000,34.080002,15.170000
1593,30.299999,33.700001,15.100000


In [185]:
preds = []
for i, data in tqdm(enumerate(batch_test.to_data_list())): # Iterate over each graph in the batch

    out = list(model(data).detach().numpy())
    
    preds.append(out)

df_pred = pd.DataFrame(np.squeeze(np.array(preds)), columns = ["petr4_pred", "btg_pred", "b3_pred"])

0it [00:00, ?it/s]

100it [00:00, 1967.30it/s]


## Result

In [184]:
pd.DataFrame(np.squeeze(np.array(preds)), columns = ["petr4_pred", "btg_pred", "b3_pred"])

,petr4_pred,btg_pred,b3_pred
0,21.063160,20.158154,10.508205
1,21.146984,19.489632,10.259307
2,20.741812,19.889858,10.720746
3,21.510771,19.915897,10.636232
4,21.447708,19.813118,10.491675
...,...,...,...
95,28.291529,31.161329,12.190410
96,28.350166,30.305115,11.547741
97,28.294792,30.987820,11.910377
98,28.715916,31.234097,11.899999


In [56]:
# Step 4: Combine edge_indices and edge_weights into a list of Data objects
data_list = [Data(edge_index=edge_indices[i], edge_attr=edge_weights[i]) for i in range(len(graphs))]

In [57]:
data_list

[Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index=[2, 6], edge_attr=[6]),
 Data(edge_index

In [33]:




# Step 5: Convert data_list into a Batch for feeding into your model
batch = geom_data.Batch.from_data_list(data_list)


,Date,petr4_close,btg_close,b3_close
0,20170224,5.587599,3.654601,5.058690
1,20170301,5.712750,3.604400,5.079965
2,20170302,5.561833,3.604400,4.973579
3,20170303,5.639132,3.765042,5.000175
4,20170306,5.558152,3.855403,4.968258
...,...,...,...,...
1590,20230719,29.100000,32.889999,14.600000
1591,20230720,29.129999,33.320000,14.610000
1592,20230721,29.680000,34.080002,15.170000
1593,20230724,30.299999,33.700001,15.100000
